In [1]:
import os

os.chdir("/root/dev/playground/knowledges/3d_representation/DMTet/")
os.getcwd()

'/root/dev/playground/knowledges/3d_representation/DMTet'

In [2]:
!gpustat

6f37a742720d                Wed Apr 17 18:15:44 2024  525.89.02
[0] NVIDIA GeForce RTX 4090 | 42°C,   5 % |   944 / 24564 MB |
[1] NVIDIA GeForce RTX 4090 | 38°C,   0 % |    10 / 24564 MB |


In [3]:
import torch

torch.cuda.set_device(1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Fit DMTets

## Store a json file which includes list of gt mesh paths

In [4]:
!ls /root/hdd2/ShapeNetCoreV2/ShapeNetCore.v2/02958343/93ce8e230939dfc230714334794526d4/images

texture0.png  texture1.png  texture2.png  texture3.png


In [5]:
shapenet_root_dir = "/root/hdd2/ShapeNetCoreV2/ShapeNetCore.v2"
output_dir = "/root/hdd2/ShapeNetCoreV2/"

In [16]:
# # create a list of paths of all gt meshes and store them as a json file
# import json
# from pathlib import Path
# from tqdm.auto import tqdm

# mesh_paths = []

# for cat_dir in tqdm(Path(shapenet_root_dir).iterdir()):
#     if cat_dir.is_dir():
#         for model_dir in cat_dir.iterdir():
#             file_path = model_dir / "models" / "model_normalized.obj"
#             if file_path.exists():
#                 mesh_paths.append(str(file_path))
#             else:
#                 print(f"There's no `{file_path}`.")

# output_file = Path(output_dir) / "shapenet_gt_mesh_paths.json"
# with open(output_file, "w") as f:
#     json.dump(mesh_paths, f, indent=4)

# print(f"Mesh paths have been svaed to {output_file}.")

0it [00:00, ?it/s]

There's no `/root/hdd2/ShapeNetCoreV2/ShapeNetCore.v2/02958343/93ce8e230939dfc230714334794526d4/models/model_normalized.obj`.
There's no `/root/hdd2/ShapeNetCoreV2/ShapeNetCore.v2/02958343/4ddef66f32e1902d3448fdcb67fe08ff/models/model_normalized.obj`.
There's no `/root/hdd2/ShapeNetCoreV2/ShapeNetCore.v2/02958343/8070747805908ae62a9eb0f146e94477/models/model_normalized.obj`.
There's no `/root/hdd2/ShapeNetCoreV2/ShapeNetCore.v2/02958343/7aa9619e89baaec6d9b8dfa78596b717/models/model_normalized.obj`.
There's no `/root/hdd2/ShapeNetCoreV2/ShapeNetCore.v2/02958343/2307b51ca7e4a03d30714334794526d4/models/model_normalized.obj`.
There's no `/root/hdd2/ShapeNetCoreV2/ShapeNetCore.v2/02958343/e6c22be1a39c9b62fb403c87929e1167/models/model_normalized.obj`.
There's no `/root/hdd2/ShapeNetCoreV2/ShapeNetCore.v2/02958343/9fb1d03b22ecac5835da01f298003d56/models/model_normalized.obj`.
There's no `/root/hdd2/ShapeNetCoreV2/ShapeNetCore.v2/02958343/3c33f9f8edc558ce77aa0b62eed1492/models/model_normalized

## Load Data

In [4]:
class ShapeNetDataset(object):
    def __init__(self, shapenet_json):
        with open(shapenet_json, 'r') as f:
            self.mesh_list = json.load(f)
        
        print(f"len all data: {len(self.mesh_list)}")

    def __len__(self):
        return len(self.mesh_list)


    def __getitem__(self, idx):
        return self.mesh_list[idx]

In [5]:
import torch

class Dataset(torch.utils.data.Dataset):
    """Basic dataset interface"""
    def __init__(self): 
        super().__init__()

    def __len__(self):
        raise NotImplementedError

    def __getitem__(self):
        raise NotImplementedError

    def collate(self, batch):
        iter_res, iter_spp = batch[0]['resolution'], batch[0]['spp']
        res_dict = {
                'mv' : torch.cat(list([item['mv'] for item in batch]), dim=0),
                'mvp' : torch.cat(list([item['mvp'] for item in batch]), dim=0),
                'campos' : torch.cat(list([item['campos'] for item in batch]), dim=0),
                'resolution' : iter_res,
                'spp' : iter_spp,
                'img' : torch.cat(list([item['img'] for item in batch]), dim=0)
            }

        if 'spts' in batch[0]:
            res_dict['spts'] = batch[0]['spts']
        if 'vpts' in batch[0]:
            res_dict['vpts'] = batch[0]['vpts']
        if 'faces' in batch[0]:
            res_dict['faces'] = batch[0]['faces']
        if 'rast_triangle_id' in batch[0]:
            res_dict['rast_triangle_id'] = batch[0]['rast_triangle_id']
        
        if 'depth' in batch[0]:
            res_dict['depth'] = torch.cat(list([item['depth'] for item in batch]), dim=0)
        if 'normal' in batch[0]:
            res_dict['normal'] = torch.cat(list([item['normal'] for item in batch]), dim=0)
        if 'geo_normal' in batch[0]:
            res_dict['geo_normal'] = torch.cat(list([item['geo_normal'] for item in batch]), dim=0)
        if 'geo_viewdir' in batch[0]:
            res_dict['geo_viewdir'] = torch.cat(list([item['geo_viewdir'] for item in batch]), dim=0)
        if 'pos' in batch[0]:
            res_dict['pos'] = torch.cat(list([item['pos'] for item in batch]), dim=0)
        if 'mask' in batch[0]:
            res_dict['mask'] = torch.cat(list([item['mask'] for item in batch]), dim=0)
        if 'mask_cont' in batch[0]:
            res_dict['mask_cont'] = torch.cat(list([item['mask_cont'] for item in batch]), dim=0)
        if 'envlight_transform' in batch[0]:
            if batch[0]['envlight_transform'] is not None:
                res_dict['envlight_transform'] = torch.cat(list([item['envlight_transform'] for item in batch]), dim=0)
            else:
                res_dict['envlight_transform'] = None

        try:
            res_dict['depth_second'] = torch.cat(list([item['depth_second'] for item in batch]), dim=0)
        except:
            pass
        try:
            res_dict['normal_second'] = torch.cat(list([item['normal_second'] for item in batch]), dim=0)
        except:
            pass
        try:
            res_dict['img_second'] = torch.cat(list([item['img_second'] for item in batch]), dim=0)
        except:
            pass


        return res_dict

In [6]:
'''from https://github.com/NVIDIAGameWorks/kaolin/blob/master/kaolin/ops/mesh/trianglemesh.py'''
# kaolin.ops.mesh.sample_points(self.ref_mesh.v_pos.unsqueeze(0), self.ref_mesh.t_pos_idx, 50000)[0][0]
def _base_face_areas(face_vertices_0, face_vertices_1, face_vertices_2):
    """Base function to compute the face areas."""
    x1, x2, x3 = torch.split(face_vertices_0 - face_vertices_1, 1, dim=-1)
    y1, y2, y3 = torch.split(face_vertices_1 - face_vertices_2, 1, dim=-1)

    a = (x2 * y3 - x3 * y2) ** 2
    b = (x3 * y1 - x1 * y3) ** 2
    c = (x1 * y2 - x2 * y1) ** 2
    areas = torch.sqrt(a + b + c) * 0.5

    return areas

def _base_sample_points_selected_faces(face_vertices, face_features=None):
    """Base function to sample points over selected faces.
       The coordinates of the face vertices are interpolated to generate new samples.

    Args:
        face_vertices (tuple of torch.Tensor):
            Coordinates of vertices, corresponding to selected faces to sample from.
            A tuple of 3 entries corresponding to each of the face vertices.
            Each entry is a torch.Tensor of shape :math:`(\\text{batch_size}, \\text{num_samples}, 3)`.
        face_features (tuple of torch.Tensor, Optional):
            Features of face vertices, corresponding to selected faces to sample from.
            A tuple of 3 entries corresponding to each of the face vertices.
            Each entry is a torch.Tensor of shape
            :math:`(\\text{batch_size}, \\text{num_samples}, \\text{feature_dim})`.

    Returns:
        (torch.Tensor, torch.Tensor):
            Sampled point coordinates of shape :math:`(\\text{batch_size}, \\text{num_samples}, 3)`.
            Sampled points interpolated features of shape
            :math:`(\\text{batch_size}, \\text{num_samples}, \\text{feature_dim})`.
            If `face_vertices_features` arg is not specified, the returned interpolated features are None.
    """

    face_vertices0, face_vertices1, face_vertices2 = face_vertices

    sampling_shape = tuple(int(d) for d in face_vertices0.shape[:-1]) + (1,)
    # u is proximity to middle point between v1 and v2 against v0.
    # v is proximity to v2 against v1.
    #
    # The probability density for u should be f_U(u) = 2u.
    # However, torch.rand use a uniform (f_X(x) = x) distribution,
    # so using torch.sqrt we make a change of variable to have the desired density
    # f_Y(y) = f_X(y ^ 2) * |d(y ^ 2) / dy| = 2y
    u = torch.sqrt(torch.rand(sampling_shape,
                              device=face_vertices0.device,
                              dtype=face_vertices0.dtype))

    v = torch.rand(sampling_shape,
                   device=face_vertices0.device,
                   dtype=face_vertices0.dtype)
    w0 = 1 - u
    w1 = u * (1 - v)
    w2 = u * v

    points = w0 * face_vertices0 + w1 * face_vertices1 + w2 * face_vertices2

    features = None
    if face_features is not None:
        face_features0, face_features1, face_features2 = face_features
        features = w0 * face_features0 + w1 * face_features1 + \
            w2 * face_features2

    return points, features

def sample_points(vertices, faces, num_samples, areas=None, face_features=None):
    r"""Uniformly sample points over the surface of triangle meshes.

    First face on which the point is sampled is randomly selected,
    with the probability of selection being proportional to the area of the face.
    then the coordinate on the face is uniformly sampled.

    If ``face_features`` is defined for the mesh faces,
    the sampled points will be returned with interpolated features as well,
    otherwise, no feature interpolation will occur.

    Args:
        vertices (torch.Tensor):
            The vertices of the meshes, of shape
            :math:`(\text{batch_size}, \text{num_vertices}, 3)`.
        faces (torch.LongTensor):
            The faces of the mesh, of shape :math:`(\text{num_faces}, 3)`.
        num_samples (int):
            The number of point sampled per mesh.
        areas (torch.Tensor, optional):
            The areas of each face, of shape :math:`(\text{batch_size}, \text{num_faces})`,
            can be preprocessed, for fast on-the-fly sampling,
            will be computed if None (default).
        face_features (torch.Tensor, optional):
            Per-vertex-per-face features, matching ``faces`` order,
            of shape :math:`(\text{batch_size}, \text{num_faces}, 3, \text{feature_dim})`.
            For example:

                1. Texture uv coordinates would be of shape
                   :math:`(\text{batch_size}, \text{num_faces}, 3, 2)`.
                2. RGB color values would be of shape
                   :math:`(\text{batch_size}, \text{num_faces}, 3, 3)`.

            When specified, it is used to interpolate the features for new sampled points.

    See also:
        :func:`~kaolin.ops.mesh.index_vertices_by_faces` for conversion of features defined per vertex
        and need to be converted to per-vertex-per-face shape of :math:`(\text{num_faces}, 3)`.

    Returns:
        (torch.Tensor, torch.LongTensor, (optional) torch.Tensor):
            the pointclouds of shape :math:`(\text{batch_size}, \text{num_samples}, 3)`,
            and the indexes of the faces selected,
            of shape :math:`(\text{batch_size}, \text{num_samples})`.

            If ``face_features`` arg is specified, then the interpolated features of sampled points of shape
            :math:`(\text{batch_size}, \text{num_samples}, \text{feature_dim})` are also returned.
    """
    if faces.shape[-1] != 3:
        raise NotImplementedError("sample_points is only implemented for triangle meshes")
    faces_0, faces_1, faces_2 = torch.split(faces, 1, dim=1)        # (num_faces, 3) -> tuple of (num_faces,)
    face_v_0 = torch.index_select(vertices, 1, faces_0.reshape(-1))  # (batch_size, num_faces, 3)
    face_v_1 = torch.index_select(vertices, 1, faces_1.reshape(-1))  # (batch_size, num_faces, 3)
    face_v_2 = torch.index_select(vertices, 1, faces_2.reshape(-1))  # (batch_size, num_faces, 3)

    if areas is None:
        areas = _base_face_areas(face_v_0, face_v_1, face_v_2).squeeze(-1)
    face_dist = torch.distributions.Categorical(areas)
    face_choices = face_dist.sample([num_samples]).transpose(0, 1)
    _face_choices = face_choices.unsqueeze(-1).repeat(1, 1, 3)
    v0 = torch.gather(face_v_0, 1, _face_choices)  # (batch_size, num_samples, 3)
    v1 = torch.gather(face_v_1, 1, _face_choices)  # (batch_size, num_samples, 3)
    v2 = torch.gather(face_v_2, 1, _face_choices)  # (batch_size, num_samples, 3)
    face_vertices_choices = (v0, v1, v2)

    # UV coordinates are available, make sure to calculate them for sampled points as well
    face_features_choices = None
    if face_features is not None:
        feat_dim = face_features.shape[-1]
        # (num_faces, 3) -> tuple of (num_faces,)
        _face_choices = face_choices[..., None, None].repeat(1, 1, 3, feat_dim)
        face_features_choices = torch.gather(face_features, 1, _face_choices)
        face_features_choices = tuple(
            tmp_feat.squeeze(2) for tmp_feat in torch.split(face_features_choices, 1, dim=2))

    points, point_features = _base_sample_points_selected_faces(
        face_vertices_choices, face_features_choices)

    if point_features is not None:
        return points, face_choices, point_features
    else:
        return points, face_choices

In [7]:
import numpy as np
import torch
import sys

from render import util
from render import mesh
from render import render
from render import light

###############################################################################
# Reference dataset using mesh & rendering
###############################################################################

class DatasetMesh(Dataset):

    def __init__(self, ref_mesh, glctx, cam_radius, FLAGS, validate=False):
        # Init 
        self.glctx              = glctx
        self.cam_radius         = cam_radius
        self.FLAGS              = FLAGS
        self.validate           = validate
        self.fovy               = np.deg2rad(45)
        self.aspect             = FLAGS.train_res[1] / FLAGS.train_res[0]
        self.random_lgt         = FLAGS.random_lgt
        self.camera_lgt         = False
        self.flat_shading       = FLAGS.dataset_flat_shading
        

        if self.FLAGS.local_rank == 0:
            print(f"use flag shading {FLAGS.dataset_flat_shading}")
            print("DatasetMesh: ref mesh has %d triangles and %d vertices" % (ref_mesh.t_pos_idx.shape[0], ref_mesh.v_pos.shape[0]))

        # Sanity test training texture resolution
        ref_texture_res = np.maximum(ref_mesh.material['kd'].getRes(), ref_mesh.material['ks'].getRes())
        if 'normal' in ref_mesh.material:
            ref_texture_res = np.maximum(ref_texture_res, ref_mesh.material['normal'].getRes())
        if self.FLAGS.local_rank == 0 and FLAGS.texture_res[0] < ref_texture_res[0] or FLAGS.texture_res[1] < ref_texture_res[1]:
            print("---> WARNING: Picked a texture resolution lower than the reference mesh [%d, %d] < [%d, %d]" % (FLAGS.texture_res[0], FLAGS.texture_res[1], ref_texture_res[0], ref_texture_res[1]))

        print("Loading env map")
        sys.stdout.flush()
        # Load environment map texture
        self.envlight = light.load_env(FLAGS.envmap, scale=FLAGS.env_scale)
        
        print("Computing tangents")
        sys.stdout.flush()
        try:
            self.ref_mesh = mesh.compute_tangents(ref_mesh)
        except Exception as e:
            print(e)
            print("Continue without tangents...")
            self.ref_mesh = ref_mesh

    def _rotate_scene(self, itr):
        proj_mtx = util.perspective(self.fovy, self.FLAGS.display_res[1] / self.FLAGS.display_res[0], self.FLAGS.cam_near_far[0], self.FLAGS.cam_near_far[1])

        # Smooth rotation for display.
        ang    = (itr / 50) * np.pi * 2
        mv     = util.translate(0, 0, -self.cam_radius) @ (util.rotate_x(-0.4) @ util.rotate_y(ang))
        mvp    = proj_mtx @ mv
        campos = torch.linalg.inv(mv)[:3, 3]

        return mv[None, ...].cuda(), mvp[None, ...].cuda(), campos[None, ...].cuda(), self.FLAGS.display_res, self.FLAGS.spp

    def _random_scene(self):
        # ==============================================================================================
        #  Setup projection matrix
        # ==============================================================================================
        iter_res = self.FLAGS.train_res
        proj_mtx = util.perspective(self.fovy, iter_res[1] / iter_res[0], self.FLAGS.cam_near_far[0], self.FLAGS.cam_near_far[1])

        # ==============================================================================================
        #  Random camera & light position
        # ==============================================================================================

        # Random rotation/translation matrix for optimization.
        mv     = util.translate(0, 0, -self.cam_radius) @ util.random_rotation_translation(0.2)
        mvp    = proj_mtx @ mv
        campos = torch.linalg.inv(mv)[:3, 3]

        return mv[None, ...].cuda(), mvp[None, ...].cuda(), campos[None, ...].cuda(), iter_res, self.FLAGS.spp # Add batch dimension

    def __len__(self):
        return 50 if self.validate else (self.FLAGS.iter + 1) * self.FLAGS.batch

    def __getitem__(self, itr):
        # ==============================================================================================
        #  Randomize scene parameters
        # ==============================================================================================

        if self.validate:
            mv, mvp, campos, iter_res, iter_spp = self._rotate_scene(itr)
            camera_mv = None
        else:
            mv, mvp, campos, iter_res, iter_spp = self._random_scene()
            if self.random_lgt:
                rnd_rot = util.random_rotation()
                camera_mv = rnd_rot.unsqueeze(0).clone()
            elif self.camera_lgt:
                camera_mv = mv.clone()
            else:
                camera_mv = None



        with torch.no_grad():
            render_out = render.render_mesh(self.glctx, self.ref_mesh, mvp, campos, self.envlight, iter_res, spp=iter_spp, 
                                num_layers=self.FLAGS.layers, msaa=True, background=None, xfm_lgt=camera_mv, flat_shading=self.flat_shading)
            img = render_out['shaded']
            img_second = render_out['shaded_second']
            normal = render_out['normal']
            depth = render_out['depth']
            geo_normal = render_out['geo_normal']
            pos = render_out['pos']

            sample_points = torch.tensor(kaolin.ops.mesh.sample_points(self.ref_mesh.v_pos.unsqueeze(0), self.ref_mesh.t_pos_idx, 50000)[0][0])
            vertex_points = self.ref_mesh.v_pos
        
        return_dict = {
            'mv' : mv,
            'mvp' : mvp,
            'campos' : campos,
            'resolution' : iter_res,
            'spp' : iter_spp,
            'img' : img,
            'img_second' : img_second,
            'spts': sample_points,
            'vpts': vertex_points,
            'faces': self.ref_mesh.t_pos_idx,
            'depth': depth,
            'normal': normal,
            'geo_normal': geo_normal,
            'geo_viewdir': render_out['geo_viewdir'],
            'pos': pos,
            'envlight_transform': camera_mv,
            'mask': render_out['mask'],
            'mask_cont': render_out['mask_cont'],
            'rast_triangle_id': render_out['rast_triangle_id']
        }

        try:
            return_dict['depth_second'] = render_out['depth_second']
        except:
            pass

        try:
            return_dict['normal_second'] = render_out['normal_second']
        except:
            pass
        return return_dict

In [8]:
from render import texture

mtl_override = None
mtl_default = {
    'name' : '_default_mat',
    'bsdf': 'diffuse',
    'uniform': True,
    'kd'   : texture.Texture2D(torch.tensor([0.75, 0.3, 0.6], dtype=torch.float32, device='cuda'), trainable=False),
    'ks'   : texture.Texture2D(torch.tensor([0.0, 0.0, 0.0], dtype=torch.float32, device='cuda'), trainable=False)
}
normal_only = True

In [9]:
!ls /root/hdd2/ShapeNetCoreV2/

ShapeNetCore.v2		     visualize
ShapeNetCore.v2.PC15k	     volume_10M
no_mtl_objs		     volume_200K
preprocessed_captions.csv    volume_500K
render_for_fid.tar.gz	     volume_6M
render_for_fid_20_views      volume_occupancies_airplane
render_for_fid_no_light      volume_occupancies_car
render_for_gen_desc	     volume_occupancies_chair
shapenet_gt_mesh_paths.json  volume_occupancies_table
split			     watertight
split_pc


In [10]:
import json
from render import mesh

shapenet_dataset = ShapeNetDataset("/root/hdd2/ShapeNetCoreV2/shapenet_gt_mesh_paths.json")
mesh_fname = shapenet_dataset[0]
mtl_override = None
ref_mesh = mesh.load_mesh(mesh_fname, mtl_override, mtl_default, use_default=normal_only, no_additional=True)

len all data: 52472
Load use-defined default mtl
Skip loading non-default materials
[{'name': '_default_mat', 'bsdf': 'diffuse', 'uniform': True, 'kd': Texture2D(), 'ks': Texture2D()}]


In [11]:
mesh_fname

'/root/hdd2/ShapeNetCoreV2/ShapeNetCore.v2/03211117/e6bedde4571506491fde762a4c6848dd/models/model_normalized.obj'

In [12]:
ref_mmesh = mesh.center_by_reference(ref_mesh, mesh.aabb_clean(ref_mesh), 1.0)

normalization: tensor([-0.0713,  0.1578, -0.0310], device='cuda:1') 1.238707634872645


In [13]:
a = ref_mesh.v_nrm.clone()
ref_mesh = mesh.auto_normals(ref_mesh)

In [14]:
import yaml

RADIUS = 2.0

# configs only related to the dataset
configs = """
    tarin_res: (512, 512)
    texture_res: (1024, 1024)
    random_lgt: True
    dataset_flat_shading: False
    local_rank: 0 # TODO: delete
    env_map: Null
    env_scale: 1.
    display_res: Null
    cam_near_far: (0.1, 1000.0)
    spp: 1
    iter: 5000
    batch: 1
    layers: 1
"""
configs = yaml.safe_load(configs)

In [ ]:
import nvdiffrast

glctx = nvdiffrast.torch.RasterizeGLContext()
dataset_train = DatasetMesh(ref_mesh, glctx, FLAGS=configs, validate=False)
dataset_val = DAtasetMesh(ref_mesh, glctx, FLAGS-configs, validate=True)

# Fitting DMTets

### Get geometry

In [5]:
# DMTetGeometry in MeshDiffusion/nvdiffrec/lib/geometry/dmtet.py
geometry = DMTetGeometry(
    model_config.dmtet_grid, model_config.mesh_scale, model_config, deform_scale=model_config.first_stage_deform
)

### Get textures

In [ ]:
from render import texture, mlptexture, material

###############################################################################
# Utility functions for material
###############################################################################

def initial_guess_material(geometry, mlp, FLAGS, init_mat=None):
    kd_min, kd_max = torch.tensor(FLAGS.kd_min, dtype=torch.float32, device='cuda'), torch.tensor(FLAGS.kd_max, dtype=torch.float32, device='cuda')
    ks_min, ks_max = torch.tensor(FLAGS.ks_min, dtype=torch.float32, device='cuda'), torch.tensor(FLAGS.ks_max, dtype=torch.float32, device='cuda')
    nrm_min, nrm_max = torch.tensor(FLAGS.nrm_min, dtype=torch.float32, device='cuda'), torch.tensor(FLAGS.nrm_max, dtype=torch.float32, device='cuda')
    if mlp:
        mlp_min = torch.cat((kd_min[0:3], ks_min, nrm_min), dim=0)
        mlp_max = torch.cat((kd_max[0:3], ks_max, nrm_max), dim=0)
        mlp_map_opt = mlptexture.MLPTexture3D(geometry.getAABB(), channels=9, min_max=[mlp_min, mlp_max])
        mat =  material.Material({'kd_ks_normal' : mlp_map_opt})
    else:
        # Setup Kd (albedo) and Ks (x, roughness, metalness) textures
        if FLAGS.random_textures or init_mat is None:
            num_channels = 4 if FLAGS.layers > 1 else 3
            kd_init = torch.rand(size=FLAGS.texture_res + [num_channels], device='cuda') * (kd_max - kd_min)[None, None, 0:num_channels] + kd_min[None, None, 0:num_channels]
            kd_map_opt = texture.create_trainable(kd_init , FLAGS.texture_res, not FLAGS.custom_mip, [kd_min, kd_max])

            ksR = np.random.uniform(size=FLAGS.texture_res + [1], low=0.0, high=0.01)
            ksG = np.random.uniform(size=FLAGS.texture_res + [1], low=ks_min[1].cpu(), high=ks_max[1].cpu())
            ksB = np.random.uniform(size=FLAGS.texture_res + [1], low=ks_min[2].cpu(), high=ks_max[2].cpu())

            ks_map_opt = texture.create_trainable(np.concatenate((ksR, ksG, ksB), axis=2), FLAGS.texture_res, not FLAGS.custom_mip, [ks_min, ks_max])
        else:
            kd_map_opt = texture.create_trainable(init_mat['kd'], FLAGS.texture_res, not FLAGS.custom_mip, [kd_min, kd_max])
            ks_map_opt = texture.create_trainable(init_mat['ks'], FLAGS.texture_res, not FLAGS.custom_mip, [ks_min, ks_max])

        # Setup normal map
        if FLAGS.random_textures or init_mat is None or 'normal' not in init_mat:
            normal_map_opt = texture.create_trainable(np.array([0, 0, 1]), FLAGS.texture_res, not FLAGS.custom_mip, [nrm_min, nrm_max])
        else:
            normal_map_opt = texture.create_trainable(init_mat['normal'], FLAGS.texture_res, not FLAGS.custom_mip, [nrm_min, nrm_max])

        mat = material.Material({
            'kd'     : kd_map_opt,
            'ks'     : ks_map_opt,
            'normal' : normal_map_opt
        })

    if init_mat is not None:
        mat['bsdf'] = init_mat['bsdf']
    else:
        mat['bsdf'] = 'pbr'

    return mat

def initial_guess_material_knownkskd(geometry, mlp, FLAGS, init_mat=None):
    nrm_min, nrm_max = torch.tensor(FLAGS.nrm_min, dtype=torch.float32, device='cuda'), torch.tensor(FLAGS.nrm_max, dtype=torch.float32, device='cuda')

    if mlp:
        mlp_min = nrm_min
        mlp_max = nrm_max
        mlp_map_opt = mlptexture.MLPTexture3D(geometry.getAABB(), channels=3, min_max=[mlp_min, mlp_max])
        # mlp_map_opt = mlptexture.MLPTexture3D(geometry.getAABB(), channels=3, min_max=None)
        mat =  material.Material({
            'kd'     : init_mat['kd'],
            'ks'     : init_mat['ks'],
            'normal' : mlp_map_opt,
        })
    else:
        # Setup normal map
        if FLAGS.random_textures or init_mat is None or 'normal' not in init_mat:
            normal_map_opt = texture.create_trainable(np.array([0, 0, 1]), FLAGS.texture_res, not FLAGS.custom_mip, [nrm_min, nrm_max])
        else:
            normal_map_opt = texture.create_trainable(init_mat['normal'], FLAGS.texture_res, not FLAGS.custom_mip, [nrm_min, nrm_max])

        mat = material.Material({
            'kd'     : init_mat['kd'],
            'ks'     : init_mat['ks'],
            'normal' : normal_map_opt
        })

    if init_mat is not None:
        mat['bsdf'] = init_mat['bsdf']
    else:
        mat['bsdf'] = 'pbr'

    return mat

In [ ]:
# set textures, make initial guess from reference if possible
mat = initial_guess_material_knownkskd(geometry, True, mat_config, mtl_default)

### Train

In [ ]:
# pass 1
geometry, mat = optimize_mesh(glctx, geometry, mat, lgt, dataset_train, dataset_val, config,
                             pass_idx=0, pass_name="dmtet_pass1", optimize_light=config.learn_light)
base_mesh = geometry.getMesh(mat)
vert_mask = torch.zeros_like(geoemtry.sdf).long().cuda().view(-1, 1)
vert_mask[geometry.getValidVertsIdx()] = 1

torch.cuda.empty_cache() # may slow down training

torch.save({
    "sdf": geometry.sdf.cpu().detach(),
    "sdf_ema": geometry.sdf_ema.cpu().detach(),
    "deform": (geometry.deform * vert_mask).cpu().detach(),
    "deform_unmasked": geometry.deform.cpu().detach(),
}, os.path.join(config.out_dir, "tets_pre/dmt_dict_{:05d}.pt".format(global_index)))
old_geometry = geometry

In [ ]:
# pass 2
geometry = DMTetGeometryFixedTopo(
    geometry, base_mesh, config.dmtet_grid, config.mesh_scale, config, deform_scale=config.second_stage_deform
)

geometry.sdf_sign.requires_grad = False
geometry.sdf_abs.requires_grad = False
geometry.deform.requires_grad = True

geometry.deform.data[:] = geometry.deform * config.first_stage_deform / config.second_stage_deform

if config.use_ema:
    geometry.sdf_sign.data[:] = torch.sign(old_geoemtry.sdf_ema)
else:
    geometry.sdf_sign.data[:] = torch.sign(old_geometry.sdf)

geometry.set_init_v_pos()

geometry, mat = optimize_mesh(glctx, geometry, mat, lgt, dataset_train, dataset_val, config,
                             pass_idx=1, pass_name="mesh_pass", warmup_iter=100,
                             optimize_light=config.learn_light and not config.lock_light,
                             optimize_geometry=not config.lock_pos)
vert_mask = torch.zeros_like(geometry.sdf_sign).long().cuda().view(-1, 1)
vert_mask[geometry.getValidVertsIdx()] = 1

torch.save({
    "sdf": geometry.sdf_sign.cpu().detach(),
    "deform": (geometry.deform * vert_mask).cpu().detach(),
    "deform_unmasked": geometry.deform.cpu().detach()
}, os.path.join(config.out_dir, "tets/dmt_dict_{:05d}.pt".format(global_index)))

# ... validation ...

# Training Diffusion Model